In [1]:
import pandas as pd
import warnings
import re
import numpy as np
import time
import PyPDF2
from PyPDF2 import PdfReader
import tabula
import os

In [2]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'
warnings.simplefilter(action='ignore', category=FutureWarning)
term_size = os.get_terminal_size()

### Select Scouts

Choose scouts

In [3]:
def clean_scouts():
    scout_assemble = pd.read_csv("Scouts/scouted this week.csv")
    scout_assemble.drop(list(scout_assemble.filter(regex='Unnam')), axis=1, inplace=True)
    scout_assemble["Country"] = scout_assemble["TEA Random"].str.split().str[-1] # get the country and add to new col
    scout_assemble["TEA Random"] = scout_assemble["TEA Random"].str.rpartition()[0] # remove the country from original col
    scout_assemble['TEA Random'] = scout_assemble['TEA Random'].str[4:] # remove "TEA "
    scout_assemble['TEA Random'] = scout_assemble['TEA Random'].str[:-2] # remove " -" from end
    #read_scoutable["TEA Random"] = read_scoutable["TEA Random"].str.rpartition()[0]
    scout_assemble = scout_assemble.rename(columns={'TEA Random': 'Team'})

    global search_MDS_4_scouts
    search_MDS_4_scouts = scout_assemble["Team"].tolist()
clean_scouts()

Import previously scouted teams and clean up

In [4]:
def filter_scouts(search,return_scouts): # set function to take two variables (these will be return_scouts and search) >> see search_scouted
    print('\n')
    print('=' * term_size.columns)
    return_scouts.drop(list(return_scouts.filter(regex='Unnamed: 0')), axis=1, inplace=True)
    return_scouts.rename(columns = { "Unnamed: 1" : "Nationality" },inplace=True) # rename column
    print('\033[1m' + search + '\n') # print team name in bold
    display(return_scouts) # show the scouted teams
    print('\n')
    print('=' * term_size.columns)

In [6]:
import matplotlib.pyplot as plt
display(search_MDS_4_scouts)
file = pd.read_csv("Turn Data/Turn Data.csv").Filepath.squeeze() # get file path 

s_PV = 32 #= int(input("What PV threshold would you like to scout for? "))
def search_scouted(result):

    pdf_file = open(file, 'rb') # Open the PDF file
    pdf_reader = PyPDF2.PdfReader(pdf_file) # Create a PDF reader object
    search = result # set the list element as the text to search by
    
    for page_number in range(len(pdf_reader.pages)): # for each page in the range
            text = pdf_reader.pages[page_number].extract_text() # extract text of PDF page
        
            if search in text: # if list element is in the page text
                page = page_number + 1 # set page number 
                st = pd.DataFrame(tabula.read_pdf(file, pages=page)[0]).replace(np.nan, '') 
                
                if st.iloc[:1,:1].squeeze() == "Goalkeepers": # ensure what we're selecting is an MDS team sheet, not something else 
                    
                    # --------- filter and apply ---------- #
                    print('\033[1m\n')
                    print('=' * term_size.columns)
                    st.drop(list(st.filter(regex='Unnamed: 0')), axis=1, inplace=True)
                    st.rename(columns = { "Unnamed: 1" : "Nationality" },inplace=True)
                    print('\033[1m\n' + search + '\n')
                    print('=' * term_size.columns)
                    #display(st) # show the scouted teams
                    #print('\n')

                    # --------- strip special characters ---------- #
                    st.iloc[:,3:16] = st.iloc[:,3:16].map(lambda x: x.rstrip('*+-')) # 

                     # --------- break into groups ---------- #
                    s = st.eq(st.columns)
                    s_groups = [g.reset_index(drop=True) for _, g in st[~s.iloc[:, 0]].groupby(s.cumsum()[~s.iloc[:, 0]].iloc[:, 0])]
                    s_gks = s_groups[0].drop(index=0)
                    s_deff = s_groups[1].drop(index=0)
                    s_mid = s_groups[2].drop(index=0)
                    s_att = s_groups[3].drop(index=0)

                    # --------- rename cols ---------- #
                    s_deff = s_deff.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref':'Jud','Crs':'Vis'})
                    s_mid = s_mid.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref':'Jud','Crs':'Vis'})
                    s_att = s_att.rename(columns={'Dis': 'Agg', 'Han': 'Tac', 'Ref':'Jud','Crs':'Vis'})

                    # --------- split ID and names ---------- #
                    s_gks[['ID', 'Name']] = s_gks['ID Name'].str.split(' ', n=1, expand=True)
                    s_gks.drop(columns="ID Name",axis=1, inplace=True)
                    s_gks = s_gks[["ID", "Name", "Age", "Foot", "Sho", "Mov","Pas","Con","Spe","Sta","Hea","Str","Dis","Han","Ref","Crs","OA","SA", "Nationality", "Price"]]


                    
                    # --------- Calculate PV ---------- #
                    s_gks['GK'] = s_gks[['Dis', 'Han', 'Ref','Crs']].astype(int).sum(axis = 1, skipna = True)
                    
                    s_deff['CB'] = s_deff[['Hea', 'Str', 'Tac','Jud']].astype(int).sum(axis = 1, skipna = True)
                    s_deff['FB'] = s_deff[['Spe', 'Sta', 'Agg','Tac']].astype(int).sum(axis = 1, skipna = True)
                    s_deff['SW'] = s_deff[['Pas', 'Con', 'Tac','Jud']].astype(int).sum(axis = 1, skipna = True)
                    s_deff['WB'] = s_deff[['Mov', 'Pas', 'Spe','Sta']].astype(int).sum(axis = 1, skipna = True)

                    s_mid['CM'] = s_mid[['Pas', 'Sta', 'Hea','Tac']].astype(int).sum(axis = 1, skipna = True)
                    s_mid['AM'] = s_mid[['Str', 'Agg', 'Tac','Jud']].astype(int).sum(axis = 1, skipna = True)
                    s_mid['PL'] = s_mid[['Pas', 'Con', 'Jud','Vis']].astype(int).sum(axis = 1, skipna = True)
                    s_mid['FR'] = s_mid[['Sho', 'Mov', 'Pas','Con']].astype(int).sum(axis = 1, skipna = True)
                    s_mid['WG'] = s_mid[['Pas', 'Con', 'Spe','Sta']].astype(int).sum(axis = 1, skipna = True)

                    s_att['WF'] = s_att[['Sho', 'Mov', 'Con','Spe']].astype(int).sum(axis = 1, skipna = True)
                    s_att['IF'] = s_att[['Pas', 'Con', 'Jud','Vis']].astype(int).sum(axis = 1, skipna = True)
                    s_att['CF'] = s_att[['Sho', 'Mov', 'Str','Agg']].astype(int).sum(axis = 1, skipna = True)
                    s_att['TM'] = s_att[['Con', 'Hea', 'Str','Agg']].astype(int).sum(axis = 1, skipna = True)

                    # --------- don't show low PVs ---------- #
                    s_gks['GK'] = s_gks['GK'].astype(int); s_gks['GK'] = np.where(s_gks['GK'] < s_PV, "", s_gks['GK'])

                    s_deff['CB'] = s_deff['CB'].astype(int); s_deff['CB'] = np.where(s_deff['CB'] < s_PV, "", s_deff['CB'])
                    s_deff['SW'] = s_deff['SW'].astype(int); s_deff['SW'] = np.where(s_deff['SW'] < s_PV, "", s_deff['SW'])
                    s_deff['WB'] = s_deff['WB'].astype(int); s_deff['WB'] = np.where(s_deff['WB'] < s_PV, "", s_deff['WB'])
                    s_deff['FB'] = s_deff['FB'].astype(int); s_deff['FB'] = np.where(s_deff['FB'] < s_PV, "", s_deff['FB'])
                    
                    s_mid['CM'] = s_mid['CM'].astype(int); s_mid['CM'] = np.where(s_mid['CM'] < s_PV, "", s_mid['CM'])
                    s_mid['AM'] = np.where(s_mid['AM'] < s_PV, "", s_mid['AM'])
                    s_mid['PL'] = np.where(s_mid['PL'] < s_PV, "", s_mid['PL'])
                    s_mid['FR'] = np.where(s_mid['FR'] < s_PV, "", s_mid['FR'])
                    s_mid['WG'] = s_mid['WG'].astype(int); s_mid['WG'] = np.where(s_mid['WG'] < s_PV, "", s_mid['WG'])
    
                    s_att['WF'] = s_att['WF'].astype(int); s_att['WF'] = np.where(s_att['WF'] < s_PV, "", s_att['WF'])
                    s_att['IF'] = np.where(s_att['IF'] < s_PV, "", s_att['IF'])
                    s_att['TM'] = np.where(s_att['TM'] < s_PV, "", s_att['TM'])
                    s_att['CF'] = np.where(s_att['CF'] < s_PV, "", s_att['CF'])


                    # --------- Show Players by Section ---------- #
                    display("Goalkeepers",s_gks.style.background_gradient(axis=None, subset=['Spe','Dis','Crs']),
                            "Defenders",s_deff.style.background_gradient(axis=None, subset=['Spe','Agg','Vis']),
                            "Midfielders",s_mid.style.background_gradient(axis=None, subset=['Spe','Agg','Vis']),
                            "Attackers",s_att.style.background_gradient(axis=None, subset=['Spe','Agg','Vis'])
                           )
                
                else: # if the searched-for text isn't in the PDF with a first cell of "Goalkeepers" - i.e. isn't a team we've scouted for, skip it
                    pass
                    
            else: # if list element is not in the page text, do nothing
                pass
    pdf_file.close()
    
#search_scouted(i) # pass each list element to the function search_scouted()

for i in search_MDS_4_scouts: # for each element in scouts
    result = search_scouted(i) # get the element and send to function search_scouted


['Belenenses',
 'FC Porto',
 'Maritimo',
 'Estoril Praia',
 'Academico Viseu',
 'Maritimo Funchal',
 'Pacos Ferreira',
 'Vasco De Gama',
 'Alverca',
 'FC Maia',
 'Naval',
 'Farense',
 'Arouca',
 'Academica',
 'Tondela',
 'Nacional',
 'S Braga',
 'Uniao Madeira',
 'Praia',
 'Sporting Espinho',
 'Feirense',
 'Trofense',
 'Barreirense',
 'Estrela Amadora',
 'Leca',
 'Uniao Lamas',
 'Sporting Lisbon',
 'Madeira',
 'Anadia',
 'S Lisbon']




Alverca



'Goalkeepers'

,ID,Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Dis,Han,Ref,Crs,OA,SA,Nationality,Price,GK
1,KUSJ,J Kuspiosz,26,R,6,8,8,7,7,9,6,7,8,6,8,8,88,,Brazilian,"£383,648",
2,PMTP,J Pinho,29,R,6,6,6,6,6,8,7,7,9,9,8,8,86,Exp,Portuguese,"£1,038,614",34


'Defenders'

...

'Midfielders'

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Nationality,Price,CM,AM,PL,FR,WG
1,NFMB T Naylor,30,R,6,5,8,6,6,10,9,9,9,10,10,6,94,Def/Exp,English,"£12,416,193",37,38,,,
2,PPKO O Perez,21,R,7,5,9,7,10,10,10,10,7,6,4,9,94,Res,Costa Rican,"£17,012,116",35,,,,36
3,ABCU K Augustine,21,R,5,7,9,6,9,9,4,9,10,10,4,9,91,Def/Inx,South Africa,"£21,206,222",32,33,,,33
4,CAF Cafu,28,L,6,6,9,8,7,8,8,7,7,8,8,8,90,,Portuguese,"£2,444,968",33,,33,,32
5,FLOE E Ferdinand,25,R,6,9,6,6,10,9,7,7,10,5,7,8,90,Inx,English,"£1,845,254",,,,,
6,NSQY K Nesterov,32,R,9,8,9,9,7,10,5,6,5,7,7,8,90,Exp,Russian,"£924,089",,,33,35,35
7,TEIP P Teixeira,21,L,7,7,10,8,9,7,8,7,7,9,4,7,90,Inx,Portuguese,"£4,715,557",34,,,32,34
8,DKHT S De Vitis,30,L,7,8,7,8,8,7,7,7,9,6,7,8,89,Exp,Italian,"£251,614",,,,,
9,PPRE K Pedro Barbos,18,R,4,5,7,5,9,4,5,7,7,4,5,8,70,Inx,Portuguese,"£2,803",,,,,


'Attackers'

,ID Name,Age,Foot,Sho,Mov,Pas,Con,Spe,Sta,Hea,Str,Agg,Tac,Jud,Vis,OA,SA,Nationality,Price,WF,IF,CF,TM
1,CJDB T Carvalhal,24,L,8,6,7,9,7,8,10,9,9,9,6,8,96,,Portuguese,"£27,182,134",,,32,37
2,SGCG G Schettine,25,R,10,8,9,9,6,6,9,8,9,7,7,6,94,Cha,Brazilian,"£11,596,789",33,,35,35
3,JAI Jaiminho,23,B,9,9,7,7,9,9,8,7,6,8,5,6,90,Mid,Portuguese,"£4,927,563",34,,,
4,BFJY U Bakero,24,R,7,6,8,9,7,8,6,7,8,7,7,9,89,Fla,Portuguese,"£4,150,384",,33,,
5,QSBP J Quim Berto,18,R,9,9,9,9,8,4,10,8,7,3,3,7,86,Inx,Portuguese,"£4,552,828",35,,33,34


IndexError: list index out of range